In [1]:
import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter('ignore', DeprecationWarning)
#warnings.simplefilter('ignore', ConvergenceWarning)

import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import scipy.stats as stats

# Cluster 1

In [2]:
# read all the data  for morto
data_zero="Cluster_1.csv"
df0=read_csv(data_zero,index_col=0)
print(df0.shape)
df0.head(3)

(4842, 8)


,sex,bmi,diastolic,sbp,hyper,diabet,fatty,cls_train
34985,0,0.008500,0.056657,0.006088,0,NaN,NaN,1
18959,0,0.008976,0.058357,0.006796,0,NaN,NaN,1
9017,0,0.008109,0.060623,0.009344,0,NaN,NaN,1


In [3]:
df00 = df0[['hyper','diabet','fatty']]
print(df00.shape)
df00.head(1)

(4842, 3)


,hyper,diabet,fatty
34985,0,NaN,NaN


# Cluster 2

In [4]:
# read all the data  for morto
data_one="Cluster_2.csv"
df1=read_csv(data_one,index_col=0)
print(df1.shape)
df1.head(1)

(7103, 8)


,sex,bmi,diastolic,sbp,hyper,diabet,fatty,cls_train
48786,0,0.012371,0.098017,0.011327,1,0.0,1.0,2


In [5]:
df11 = df1[['hyper','diabet','fatty']]
print(df11.shape)
df11.head(2)

(7103, 3)


,hyper,diabet,fatty
48786,1,0.0,1.0
41912,1,0.0,1.0


# Cluster 3

In [6]:
# read all the data  for morto
data_two="Cluster_3.csv"
df2=read_csv(data_two,index_col=0)
print(df2.shape)
df2.head(1)

(7943, 8)


,sex,bmi,diastolic,sbp,hyper,diabet,fatty,cls_train
55703,1,0.006842,0.05949,0.009628,0,NaN,NaN,3


In [7]:
df22 = df2[['hyper','diabet','fatty']]
print(df22.shape)
df22.head(2)

(7943, 3)


,hyper,diabet,fatty
55703,0,NaN,NaN
25330,0,NaN,NaN


# Chi-Sqaure Test between clusters

#### testing Observed values for  three clusters with variable hypertension

In [8]:
df0_table = pd.crosstab(index=df00['hyper'], columns="count")
df0_table.T # tain table

hyper,0,1
col_0,,
count,2972,1870


In [9]:
df1_table = pd.crosstab(index=df11['hyper'], columns="count")
df1_table.T #Test table

hyper,0,1
col_0,,
count,3685,3418


In [10]:
df2_table = pd.crosstab(index=df22['hyper'], columns="count")
df2_table.T #Test table

hyper,0,1
col_0,,
count,2390,5553


Join tables

In [11]:
tb= pd.concat([df0_table,df1_table,df2_table],axis=1,ignore_index=True) #Join the two tables
tb.columns = ["c0","c1","c2"]
#observed = tb.ix[0:len(df0_table),0:3]
observed = tb.iloc[0:len(df0_table),0:3]
observed

,c0,c1,c2
hyper,,,
0,2972,3685,2390
1,1870,3418,5553


Calculate row total and column total

In [12]:
col_total_obs=observed.pivot_table(index=observed.index, margins=True, margins_name='col_totals', aggfunc=sum)
col_total_obs['row_totals'] = col_total_obs[observed.columns].sum(axis=1)
col_total_obs

,c0,c1,c2,row_totals
hyper,,,,
0,2972,3685,2390,9047
1,1870,3418,5553,10841
col_totals,4842,7103,7943,19888


# Expected Values for the three clusters

In [13]:
expected =  np.outer(col_total_obs["row_totals"][0:len(df0_table)],col_total_obs.loc["col_totals"][0:3]) / 19888
expected = pd.DataFrame(expected)

expected.columns = ["c0","c1","c2"]
expected.index = observed.index
expected

,c0,c1,c2
hyper,,,
0,2202.613335,3231.136414,3613.250251
1,2639.386665,3871.863586,4329.749749


In [14]:
row=len(expected)-1 ;row
col=expected.shape[1]-1 ;col
tot=row*col
tot

2

Chi-Square statstic

In [15]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()
print(chi_squared_stat.round(4))

1369.7053


In [16]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df =tot)   # *

print("Critical value")
print(crit.round(4))

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=tot)
print("P value")
print(p_value.round(4))

Critical value
5.9915
P value
0.0


### Complete code 

In [19]:
df_report2=pd.DataFrame({})
#df_report = pd.DataFrame(df_report, columns = ['in','var',"samp1_n", "samp1_per", "samp2_n", "samp2_tot","Chi-sq","DF","CV","PV"])
for i in range(0,3):
    df0_table = pd.crosstab(index=df00.iloc[:,i], columns="")
    df1_table = pd.crosstab(index=df11.iloc[:,i], columns="") 
    df2_table = pd.crosstab(index=df22.iloc[:,i], columns="") 
    # join tables
    tb = pd.concat([df0_table,df1_table,df2_table],axis=1, ignore_index=True)
    tb.columns = ["c0","c1","c2"]
    # observed
    #tb.columns = ["s","p"]
    observed = tb.iloc[0:len(df0_table),0:3]
    #row total and column total
    col_total_obs=observed.pivot_table(index=observed.index, margins=True, margins_name='col_totals', aggfunc=sum)
    col_total_obs['row_totals'] = col_total_obs[observed.columns].sum(axis=1)
    col_total_obs
    #calculate expected
    expected =  np.outer(col_total_obs["row_totals"][0:len(df0_table)],col_total_obs.loc["col_totals"][0:3])/19888
    expected = pd.DataFrame(expected)
    expected.columns = ["c0","c1","c2"]
    #expected.index = observed.index
    

    # Chi-Square Test
    chi_x = (((observed-expected)**2)/expected).sum().sum()
    chi_x=pd.DataFrame([chi_x]).round(4)
    
    #Critical Value 
    row=len(expected)-1 ;
    col=expected.shape[1]-1 ;
    tot=row*col
    crit = stats.chi2.ppf(q = 0.95,df = tot)
    crit=pd.DataFrame([crit]).round(4)
    
    # P-value 
    p_value = 1 - stats.chi2.cdf(x=chi_x, df=tot)
    p_value=pd.DataFrame(p_value).round(4)
    
    #Degrees of fredom
    dfree=tot
    dfree=pd.DataFrame([dfree]).round(4)
    
    # cluster 0 (count and percentage)
    val=df00.iloc[:,i].value_counts()
    per=(100. * (df00.iloc[:,i]).value_counts() / len(df00.iloc[:,i])).round(4)
    dfn = pd.concat([val, per], keys=['', ''],axis=1, sort=True) 
    
    # cluster 1 (count and percentage)
    val=df11.iloc[:,i].value_counts()
    per=(100. * (df11.iloc[:,i]).value_counts() / len(df11.iloc[:,i])).round(4)
    dfm = pd.concat([val, per], keys=['', ''],axis=1, sort=True)
    
    # cluster 2 (count and percentage)
    val=df22.iloc[:,i].value_counts()
    per=(100. * (df22.iloc[:,i]).value_counts() / len(df22.iloc[:,i])).round(4)
    dfc = pd.concat([val, per], keys=['', ''],axis=1, sort=True)
    
    #get variable names (column names)
    col=list(df00.columns.values)
    var_name=col[i]
    var_name=pd.DataFrame([var_name])
    #
    # Total row /data size for the sample and total sample
    samp0 = "%.0f (%.2f)" % ((len(df00)),len(df00)/(len(df00)+len(df11)+len(df22)))
    samp1 = "%.0f (%.2f)" % ((len(df11)),len(df11)/(len(df00)+len(df11)+len(df22)))
    samp2 = "%.0f (%.2f)" % ((len(df22)),len(df22)/(len(df00)+len(df11)+len(df22)))
    samp0=pd.DataFrame([samp0])
    samp1=pd.DataFrame([samp1])
    samp2=pd.DataFrame([samp2])
    # Join Results
    dfk = pd.concat([var_name,dfn,samp0,dfm,samp1,dfc,samp2,chi_x,dfree,crit,p_value],axis=1, sort=True,ignore_index=True).T.drop_duplicates().T
    #dfk.reset_index()
    #dfk[1] = dfk.index #make the variables as index
    #dfk.index.rename('foo', inplace=True)
    #dfk.reset_index(level=['foo'])
    
    
    #dfk.rename(columns={0:'Num_samp',1:'Num_samp',2:'Num_tot',3:'Per_tot',4:'X^2',5:'DF',6:'CV',7:'PV'}, inplace=True)
    dfk.rename(columns={0:'',1:'',2:'',3:'',4:'',5:'',6:'',7:'',8:'',9:'',10:''}, inplace=True)
    #df_final=dfk.columns=["Num_samp", "Num_samp", "Num_tot", "Per_tot","Chi-sq","DF","CV","PV"]
    #result.append(df_final)
    df_final=dfk.fillna('')
    df_report2=df_report2.append(df_final)
    #print(df_final)
print(df_report2)
#df_report2.to_csv("C:/Progetti/selected_negative_and_positive_samples/report_detail.csv")
#df3=pd.DataFrame(result)
#print(result)

                                                                           \
0.0   hyper  2972  61.3796  4842 (0.24)  3685  51.8795  7103 (0.36)  2390   
1.0          1870  38.6204               3418  48.1205               5553   
0.0  diabet   801  16.5428  4842 (0.24)  1561  21.9766  7103 (0.36)  2412   
1.0           151   3.1185                299   4.2095                635   
0.0   fatty   347   7.1665  4842 (0.24)   683   9.6157  7103 (0.36)   706   
1.0           531  10.9665               1041  14.6558               2091   

                                        11      12   13  
0.0  30.0894  7943 (0.40)   1369.7053  2.0  5.9915  0.0  
1.0  69.9106                                             
0.0  30.3664  7943 (0.40)   9973.1047  2.0  5.9915  0.0  
1.0   7.9945                                             
0.0   8.8883  7943 (0.40)  11023.9732  2.0  5.9915  0.0  
1.0  26.3251                                             


In [20]:
writer = pd.ExcelWriter('Pvalue_detail_Km_chronic_dataset.xlsx')
df_report2.to_excel(writer,'Sheet1')
#df2.to_excel(writer,'Sheet2')
writer.save()